In [43]:
import pandas as pd
df = pd.read_csv("data/fashion.csv")
df.head()

,ProductID,Type,ProductBrand,Color,Size,LookBook,Gender,Fabric,ImageLink,Season,DiscountedPrice,Discount,ProductLink
0,1.0,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,S,Casual,men,Cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
1,1.0,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,M,Casual,men,Cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
2,1.0,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,L,Casual,men,Cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
3,1.0,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,XL,Casual,men,Cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
4,2.0,Puma Men's Regular Fit T-Shirt,Puma,Black/White,S,Sports,men,Cotton,https://m.media-amazon.com/images/I/513rGsnZbo...,NaN,"1,439",20%,https://www.amazon.in/Puma-Printed-Regular-T-S...


In [44]:
df = df.drop(columns=["ProductLink","Season" , "ImageLink" , "ProductID"])

In [45]:
df.columns

Index(['Type', 'ProductBrand', 'Color', 'Size', 'LookBook', 'Gender', 'Fabric',
       'DiscountedPrice', 'Discount'],
      dtype='object')

In [46]:
df = df.dropna()

In [47]:
df.head()

,Type,ProductBrand,Color,Size,LookBook,Gender,Fabric,DiscountedPrice,Discount
0,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,S,Casual,men,Cotton,"2,489",47%
1,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,M,Casual,men,Cotton,"2,489",47%
2,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,L,Casual,men,Cotton,"2,489",47%
3,Nike Men's Relaxed Fit T-Shirt,Nike,Gold,XL,Casual,men,Cotton,"2,489",47%
4,Puma Men's Regular Fit T-Shirt,Puma,Black/White,S,Sports,men,Cotton,"1,439",20%


In [48]:
df["DiscountedPrice"] = df["DiscountedPrice"].apply(lambda x : "price {} ".format(x))

In [49]:
df["str"] = " product name : " + df['Type'] +" company brand : "+ df["ProductBrand"] + " item color " + df["Color"] + " item size : " + df["Size"] + " style look : " + df["LookBook"] + " gender : " + df["Gender"] + " cloth fabric : " + df["Fabric"] + " discounted price : " + df["DiscountedPrice"] + " discount : " + df["Discount"]

In [50]:
df['str'][0]

" product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : S style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%"

In [51]:
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv
import requests

from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv
from langchain import OpenAI
import openai
from langchain_community.vectorstores import Milvus
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
from langchain.memory import PostgresChatMessageHistory

load_dotenv()

True

In [52]:
for i in df["str"][:10] :
    print(i)

 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : S style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : M style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : L style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : XL style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 product name : Puma Men's Regular Fit T-Shirt company brand : Puma item color Black/White item size : S style look : Sports gender : men cloth fabric : Cotton discounted price : price 1,439  discoun

In [53]:
seperator = "\n ************ \n"
data = " "
for i in df["str"]:
    # print(type(data) ,type(i) ,type(seperator) , )
    data = data + i + seperator


In [54]:
def get_docs(collection_name):
    # data = get_data(collection_name)
    text_splitter = CharacterTextSplitter(
        separator = "************",
        chunk_size=1000,
        chunk_overlap=0
    )
    dataDocument = [Document(page_content=data, metadata={"namespace": collection_name})]
    docs = text_splitter.split_documents(dataDocument)
    return docs



def create_vectorDB(collection_name):
    docs = get_docs(collection_name)
    initialize_embeddings()
    global vector_db
    vector_db = Milvus.from_documents(
        docs,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
            "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
            "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
            # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
            "secure": True,
        },
    )


def initialize_embeddings():
    try :
        global embeddings
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="sentence-transformers/all-MiniLM-l6-v2"
        )
    except Exception as e :
        print("probelm accured while creating embeddings \n{}".format(e))


In [55]:
create_vectorDB("similarity")

In [14]:
text_splitter = CharacterTextSplitter(
    separator = "************",
    chunk_size=900,
    chunk_overlap=0
)
dataDocument = [Document(page_content=data, metadata={"namespace": "simm"})]
docs = text_splitter.split_documents(dataDocument)

In [56]:
def load_vectorDB(collection_name):
    global vector_db
    vector_db = Milvus(
    embeddings,
    collection_name=collection_name,
    connection_args={
    "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
    "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
    "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
    # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
    "secure": True,
    },
)
    
load_vectorDB("similarity")


In [57]:
print(data)

  product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : S style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : M style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : L style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Nike Men's Relaxed Fit T-Shirt company brand : Nike item color Gold item size : XL style look : Casual gender : men cloth fabric : Cotton discounted price : price 2,489  discount : 47%
 ************ 
 product name : Puma Men's Regular Fit T-Shirt company brand : Puma item color Black/White item size : S style look : Sports gender : men 

In [58]:
vector_db.similarity_search(query="casual t shirt" , k=10)

[Document(page_content="product name : U.S. POLO ASSN. Men's Regular Fit T-Shirt company brand : U.S. Polo item color Maroon/Aqua/Olive/Red/Navy item size : L style look : Regular gender : men cloth fabric : Cotton discounted price : price 1,297  discount : 41%\n ************ \n product name : U.S. POLO ASSN. Men's Regular Fit T-Shirt company brand : U.S. Polo item color Maroon/Aqua/Olive/Red/Navy item size : M style look : Regular gender : men cloth fabric : Cotton discounted price : price 1,297  discount : 41%\n ************ \n product name : U.S. POLO ASSN. Men's Regular Fit T-Shirt company brand : U.S. Polo item color Maroon/Aqua/Olive/Red/Navy item size : XL style look : Regular gender : men cloth fabric : Cotton discounted price : price 1,297  discount : 41%", metadata={'namespace': 'similarity'}),
 Document(page_content="product name : CVC Shining Shirt | Formal Shirt | Clubwear Shirt company brand : CVC item color Black/White/Red/Pink item size : 2XL style look : Formal/Clubwea